In [ ]:
#VISUALIZACIÓN DE DATOS: SEABORN
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import seaborn as sns
#Cargamos datos de seguridad
ads = pd.read_csv("./Datasets/advertising.csv", sep = ";")
#preparamos los datos para que sea facil pintarlos
ads_m = ads.melt(id_vars = 'Sales',
                 value_vars = ['TV', 'Radio', 'Newspaper'], var_name='Media',
                 value_name='Spend')
#representamos en forma de rejilla las ventas segun gasto en cada medio
sns.FaceGrid(ads_m, col='Media', sharex=False).map(sns.regplot, 'Spend', 'Sales', fit_reg=False)
#preparamos los datos para que sea facil pintarlos
ads_m = ads.melt(id_vars = 'Sales',
                 value_vars = ['TV', 'Radio', 'Newspaper'], var_name='Media',
                 value_name='Spend')
#representamos en forma de rejilla las ventas segun gasto en cada medio
sns.FaceGrid(ads_m, col='Media', sharex=False).map(sns.regplot, 'Spend', 'Sales', fit_reg=False)
#modelo y ajuste de coeficientes por minimos cuadrados
import statsmodels.api as sm
#modelo (consultar ecuacion en pagina 10)
#vestor respuesta
y = ads['Sales']
#matriz de variables explicativas
X = ads['TV']
#añadimos un termino constante (b0)
X = sm.add_constant(X, prepend=False)
#construimos el objeto modelo
model = sm.OLS(y, X)
#ajustamos los parametros al modelo
mfitted = model.fit()
#resumen del modelo ajustado
print(mfitted.summary())
#grafico de valores estimados
fig = plt.figure()
fig = sm.graphics.plot_fit(mfitted, "TV").set_size_inches(10, 6)
#hacer estimaciones con nuevos datos de entrada
#generamos valores aleatorios para el gasto en TV
nuevas_obs = np.random.uniform(low=0, high=200, size=10)
#añadimos la constante
nuevas_obs = sm.add_constant(nuevas_obs, prepend=False)
#predecir la respuesta
y_pred = mfitted.predict(nuevas_obs)
#pintamos variables de gasto (X) y ventas estimadas (y_pred)
sns.regplot(x=nuevas_obs[:,0], y=y_pred, fit_reg=False)
#Otro ejemplo (ecuacion en la pagina 11)
#vector respuesta
y = ads['Sales']
#matriz de variables explicativas del modelo
X = ads[['TV', 'Radio', 'Newspaper']]
#añadimos termino constante (b0)
X = sm.add_constant(X, prepend=False)
#construimos el objeto modelo (modelo lineal tipo OLS)
modelo_mlin = sm.OLS(y, x)
#ajustamos los parametros del modelo
mlin_fitted = modelo_mlin.fit()
#resumen del modelo ajustado
print(mlin_fitted.summary())
#relacion parcial entre la variable respuesta y cada variable explicativa
fig = plt.figure()
fig = sm.graphics.plot_partregress_grid(mlin_fitted,
                                        ['TV', 'Radio', 'Newspaper']).set_size_inches(10, 6)
#definir un modelo mediante formulas
import statsmodels.formula.api as smf
#definimos el modelo usando una formula como en R
model_f = smf.ols(formula = 'Sales - TV + Radio + Newspaper',
                  data=ads)
#ajustamos el modelo
mf_fitted = model_f.fit()
#resumen
print(mf_fitted.summary())
#Variables categoricas
#creamos una variable categorica
#¿el gasto de TV es bajo, medio o alto?
ads['TV_budget'] = pd.qcut(ads['TV'], [0, 0.15, 0.85, 1], 
labels=['LOW', 'MID', 'HIGH'])
#creamos una variable entera
#0 si hay mas gasto en TV que el resto, 1 el contrario
ads['More_Radio_Or_News'] = ads.apply(lambda x: int(x['TV'] <
                                                    x['Radio'] + x['Newspaper']), axis="columns")
#definimos un modelo usando variables categoricas
model_c = smf.ols(formula = 'Sales - TV_budget + C(More_Radio_Or_News)', data=ads)
mc_fitted = model_c.fit()
print(mc_fitted.summary())
#Transformaciones. Funciones en formulas (funciones vectorizadas)
#definimos un modelo usando la reiz cuadrada del gasto en TV como predictor
model_fn = snf.ols(formula = 'Sales - np.sqrt(TV)', data=ads)
mfn_fitted = model_fn.fit()
print(mfn_fitted.summary())
#regresion logislica
body = pd.read_csv("./Datasets/body_measures_subset.csv", delimiter = ";")
body.head()
#variable respuesta
y = body['Gender']
#las variables predictoras son todas las demas
X = body.iloc[:, 1:]
#añadimos el termino constante
X = sm.add_constant(X, prepend=False)
#construimos el modelo logistico
model = smf.Logit(y, X)
#ajustamos
mfitted = model.fit()
#examinamos los resultados del ajuste
print(mfitted.summary())
#matriz de confusion (filas observaciones de la variable respuesta, columnas valores estimados del modelo ajustado)
print(mfitted.pred_table())
#dividir los datos originales (conjunto de entrenamiento y de test)
#creamos el conjunto de entrenamiento
#tomando aleatoriamente el 80% de filas
body_train = body.copy().sample(frac=0.8)
#el conjunto de test (el resto de filas)
body_test = body.copy().drop(body_train.index)
#variable respuesta
y_train = body_train['Gender']
y_test = body_test['Gender']
#las variables predictoras son todas las demas
X_train = body_train.iloc[:, 1:]
X_test = body_test.iloc[:, 1:]
#añadimos el termino constante
X_train = sm.add_constant(X_train, prepend=False)
X_test = sm.add_constant(X_test, prepend=False)
#construimos el modelo logistico
model = smf.Logit(y_train, X_train)
#ajustamos
mfitted = model.fit()
#prediccion del conjunto del test
pred = mfitted.predict(X_test)
pred.head()
#Calculamos la tabla de confusion
pred_0_1 = [1 if p >= 0.5 else 0 for p in pred]
pd.crosstab(index=y_test, columns=pd.Categorical(pred_0_1),
            rownames=['Obs'], colnames=['Pred'])
#calcular el acierto global
acierto_total = np.mean(y_test == pred_0_1)
print('acierto_total = {0}%'.format(np.round(acierto_total*100, 2)))
acierto_total = 92.08%